## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-13-01-14-00 +0000,wsj,SpaceX Tells Staff It Is Preparing for a Poten...,https://www.wsj.com/business/spacex-tells-staf...
1,2025-12-13-01-12-49 +0000,nypost,White House hosts ‘Miracle on Ice’ heroes as T...,https://nypost.com/2025/12/12/us-news/hockey-h...
2,2025-12-13-01-09-53 +0000,nyt,House G.O.P. Releases Health Plan That Would A...,https://www.nytimes.com/2025/12/12/us/politics...
3,2025-12-13-01-06-01 +0000,nypost,LAPD ‘bleeding out’ as City Hall fights over c...,https://nypost.com/2025/12/12/us-news/la-faces...
4,2025-12-13-01-00-14 +0000,nyt,"After False Earthquake Alert in Nevada, Lawmak...",https://www.nytimes.com/2025/12/12/us/usgs-nev...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,48
78,new,21
7,year,13
105,venezuela,12
9,house,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
92,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...,126
326,2025-12-12-02-38-00 +0000,wsj,President Trump signs an executive order that ...,https://www.wsj.com/politics/policy/trump-sign...,120
195,2025-12-12-15-51-53 +0000,nypost,"Woody Allen, Andrew Mountbatten-Windsor, Bill ...",https://nypost.com/2025/12/12/us-news/woody-al...,104
133,2025-12-12-19-33-00 +0000,wsj,New Epstein Photos Released by Democrats Show ...,https://www.wsj.com/politics/democrats-release...,103
22,2025-12-13-00-06-14 +0000,nyt,New Photos From Epstein Estate Show Ties to Tr...,https://www.nytimes.com/2025/12/12/us/politics...,101


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
92,126,2025-12-12-21-34-00 +0000,wsj,President Trump said he was leaning toward cho...,https://www.wsj.com/economy/central-banking/tr...
68,56,2025-12-12-22-27-39 +0000,nypost,Creepy Jeffrey Epstein bubble bath photo surfa...,https://nypost.com/2025/12/12/us-news/creepy-j...
12,50,2025-12-13-00-35-21 +0000,bbc,"As Trump targets Venezuela, its allies Russia ...",https://www.bbc.com/news/articles/c075mr3190mo...
221,43,2025-12-12-14-06-49 +0000,cbc,Montreal-Ottawa will be 1st leg of 'transforma...,https://www.cbc.ca/news/canada/livestory/high-...
99,37,2025-12-12-21-03-00 +0000,wsj,Reddit Sues Australian Government for Under-16...,https://www.wsj.com/business/media/reddit-sues...
13,37,2025-12-13-00-31-18 +0000,nyt,"Ukrainians, Worrying More About Power Cuts, Se...",https://www.nytimes.com/2025/12/12/world/europ...
35,35,2025-12-12-23-35-04 +0000,nypost,Alleged California ‘cult’ leaders arrested for...,https://nypost.com/2025/12/12/us-news/alleged-...
168,35,2025-12-12-17-34-00 +0000,wsj,Amtrak managers are giving up $16 million of t...,https://www.wsj.com/politics/policy/amtrak-wor...
165,32,2025-12-12-17-46-17 +0000,nypost,Disney CEO Bob Iger raises red flags about Net...,https://nypost.com/2025/12/12/business/disney-...
216,31,2025-12-12-14-27-32 +0000,bbc,"Disguises, freezing waves, and a special force...",https://www.bbc.com/news/articles/c4g990exxq4o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
